# Project: Generating Artistic Abstract Images with GANs
## by Ajay Sethuraman

**Introduction**

In recent years, Generative Adversarial Networks (GANs) have revolutionized the field of artificial intelligence by enabling machines to generate realistic images. This project explores the creative potential of GANs by training a model to generate unique and artistic abstract images. By leveraging deep learning techniques, this project aims to develop a GAN that learns artistic patterns and produces visually compelling results.

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.utils as vutils
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Hyperparameters
image_size = 64
batch_size = 128
latent_dim = 100
num_epochs = 100
lr = 0.0002
beta1 = 0.5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

**Dataset Preparation**

To train the GAN effectively, we used the WikiArt dataset, specifically selecting abstract art images. This publicly available dataset provides a diverse collection of artistic styles, allowing the model to learn from a rich variety of abstract compositions. The preprocessing steps included:

* Resizing images to a fixed dimension of 64×64 pixels for uniformity.
* Normalizing pixel values to the range [-1, 1] to enhance model performance.
* Loading the dataset directly using `torchvision.datasets.WikiArt`, eliminating the need for manual data collection and structuring.

<br>These steps ensured that the GAN trained on a well-organized and high-quality dataset, improving the fidelity and artistic diversity of the generated images.

In [ ]:
# Define transformations
image_size = 64  # Adjust based on model requirements
batch_size = 128  # Set an appropriate batch size

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1] range
])

# Load the WikiArt dataset
dataset = datasets.WikiArt(root="./data", category="abstract", transform=transform, download=True)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

**Model Architecture**

The GAN consists of two neural networks: a Generator and a Discriminator, both utilizing convolutional layers.

**Generator**

The Generator takes random noise as input and produces abstract images. It uses multiple convolutional transpose layers to progressively refine the output, with batch normalization and ReLU activation functions to improve stability and visual coherence.

**Discriminator**

The Discriminator’s role is to distinguish real images from the dataset and fake images generated by the Generator. It employs convolutional layers with LeakyReLU activation, batch normalization, and a final sigmoid activation function to classify images.

By training both networks simultaneously in an adversarial manner, the Generator improves its ability to create visually convincing abstract images.

In [ ]:
# Generator Model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)
# Discriminator Model
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 128, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x).view(-1, 1)

**Training Process**

The GAN was trained using an adversarial training loop:

* Training the Discriminator: The model learns to differentiate real images from generated ones by optimizing the binary cross-entropy loss.

* Training the Generator: The Generator attempts to produce images that can fool the Discriminator, minimizing its loss function.

* Monitoring Losses: The training process was carefully balanced to prevent mode collapse and ensure stable learning.

The Adam optimizer was used for both models with a learning rate of 0.0002 and beta1 of 0.5 to maintain steady convergence.

In [ ]:
# Initialize models
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss and optimizers
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))

# Training loop
for epoch in range(num_epochs):
    for i, (real_images, _) in enumerate(dataloader):
        real_images = real_images.to(device)
        batch_size = real_images.size(0)

        # Train Discriminator
        optimizer_D.zero_grad()
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        outputs = discriminator(real_images)
        d_loss_real = criterion(outputs, real_labels)
        d_loss_real.backward()

        noise = torch.randn(batch_size, latent_dim, 1, 1).to(device)
        fake_images = generator(noise)
        outputs = discriminator(fake_images.detach())
        d_loss_fake = criterion(outputs, fake_labels)
        d_loss_fake.backward()

        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        optimizer_G.step()

    print(f"Epoch [{epoch+1}/{num_epochs}] Loss D: {d_loss_real + d_loss_fake:.4f}, Loss G: {g_loss:.4f}")

    if epoch % 10 == 0:
        vutils.save_image(fake_images, f"generated_{epoch}.png", normalize=True)

**Generating and Evaluating Images**

After training, the Generator was used to create new abstract art images by feeding it random noise vectors. The generated images were saved and visually inspected to assess their artistic qualities. Evaluation criteria included:

* Visual Diversity: Checking whether the model produces a variety of styles and compositions.

* Aesthetic Appeal: Assessing the artistic qualities of the generated images.

* Similarity to Real Abstract Art: Ensuring that the outputs resemble real-world abstract art rather than noise or repetitive patterns.

In [ ]:
# Save Models
torch.save(generator.state_dict(), "generator.pth")
torch.save(discriminator.state_dict(), "discriminator.pth")

# Generate Sample Images
with torch.no_grad():
    noise = torch.randn(16, latent_dim, 1, 1).to(device)
    generated_images = generator(noise)
    vutils.save_image(generated_images, "final_generated.png", normalize=True)

**Reflections and Insights**

This project highlights the creative capabilities of GANs in generating artistic images. Throughout the development process, we observed:

The importance of proper dataset preprocessing in achieving high-quality outputs.

The necessity of balancing Discriminator and Generator training to avoid overfitting or mode collapse.

The potential for extending the project by training on specific art styles (e.g., impressionism, cubism) for more stylistic control.

**Conclusion**

This project successfully demonstrated how GANs can generate artistic abstract images by learning patterns from real-world abstract art datasets. The results illustrate the ability of AI to engage in creative tasks, expanding the applications of deep learning in digital art and design. Future enhancements could include user-controlled parameters for interactive generation, allowing for greater customization of artistic outputs.

By blending AI with artistic expression, this project underscores the evolving relationship between technology and creativity, paving the way for new forms of digital art.
